In [ ]:
import os
from groq import Groq
from openai import OpenAI
from pydub import AudioSegment
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Groq API key setup (FREE!)
API_KEY = "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"  
client = Groq(api_key="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")
OPENAI_API_KEY = "sk-AfX0rModIomRG94i_uu6EmTxDke-lTJjrd-ypIa0mVT3BlbkFJGWF5LOwnni53vOR9uvHv2FU_H98V4-DHOqp87EP1gA"  # Apni puri key yahan daalo
client = OpenAI(api_key="sk-AfX0rModIomRG94i_uu6EmTxDke-lTJjrd-ypIa0mVT3BlbkFJGWF5LOwnni53vOR9uvHv2FU_H98V4-DHOqp87EP1gA")
# Folders setup
audio_folder = "audios"
output_folder = "transcriptions"

os.makedirs(output_folder, exist_ok=True)


MAX_WORKERS = 5  
CHUNK_LENGTH_MS = 600000  
def transcribe_audio_direct(audio_path):
   
    try:
        file_size = os.path.getsize(audio_path) / (1024 * 1024)  
        
        print(f"  File size: {file_size:.2f} MB")
        
        if file_size < 24: 
            print(f"  Transcribing directly (Hindi + English)...")
            
           
            with open(audio_path, "rb") as audio_file:
                hindi_transcript = client.audio.transcriptions.create(
                    model="whisper-large-v3",
                    file=audio_file,
                    response_format="verbose_json",
                    language="hi",
                    temperature=0.0
                )
            
            # English translation
            with open(audio_path, "rb") as audio_file:
                english_transcript = client.audio.translations.create(
                    model="whisper-large-v3",
                    file=audio_file,
                    response_format="verbose_json",
                    temperature=0.0
                )
            
            return [(hindi_transcript, english_transcript)], False
        else:
            print(f"  File too large, will use chunking...")
            return None, True
            
    except Exception as e:
        print(f"  Error: {e}")
        return None, True

def split_and_transcribe_chunk(args):
 
    audio, chunk_idx, start_ms, end_ms, audio_name = args
    
    try:
        chunk = audio[start_ms:end_ms]
        temp_path = f"temp_chunk_{audio_name}_{chunk_idx}.mp3"
        chunk.export(temp_path, format="mp3")
        
        # Hindi Transcription
        with open(temp_path, "rb") as audio_file:
            hindi_transcript = client.audio.transcriptions.create(
                model="whisper-large-v3",
                file=audio_file,
                response_format="verbose_json",
                language="hi",
                temperature=0.0
            )
        
        # English Translation
        with open(temp_path, "rb") as audio_file:
            english_transcript = client.audio.translations.create(
                model="whisper-large-v3",
                file=audio_file,
                response_format="verbose_json",
                temperature=0.0
            )
        
        os.remove(temp_path)
        
        start_sec = start_ms / 1000
        if hasattr(hindi_transcript, 'segments') and hindi_transcript.segments:
            for segment in hindi_transcript.segments:
                segment['start'] += start_sec
                segment['end'] += start_sec
        
        if hasattr(english_transcript, 'segments') and english_transcript.segments:
            for segment in english_transcript.segments:
                segment['start'] += start_sec
                segment['end'] += start_sec
        
        return {
            'chunk_idx': chunk_idx,
            'hindi_transcript': hindi_transcript,
            'english_transcript': english_transcript,
            'start': start_sec,
            'success': True
        }
        
    except Exception as e:
        print(f"\n  ✗ Chunk {chunk_idx} failed: {e}")
        return {
            'chunk_idx': chunk_idx,
            'success': False,
            'error': str(e)
        }

def process_with_chunking(audio_path):
    """
    Badi files ke liye parallel chunking use karta hai
    """
    print(f"  Loading audio for chunking...")
    audio = AudioSegment.from_mp3(audio_path)
    
    total_duration_ms = len(audio)
    num_chunks = (total_duration_ms + CHUNK_LENGTH_MS - 1) // CHUNK_LENGTH_MS
    
    print(f"  Duration: {total_duration_ms/1000/60:.1f} minutes")
    print(f"  Creating {num_chunks} chunks of ~{CHUNK_LENGTH_MS/1000/60:.0f} minutes each")
    
    chunk_args = []
    audio_name = os.path.splitext(os.path.basename(audio_path))[0]
    
    for i in range(num_chunks):
        start_ms = i * CHUNK_LENGTH_MS
        end_ms = min((i + 1) * CHUNK_LENGTH_MS, total_duration_ms)
        chunk_args.append((audio, i, start_ms, end_ms, audio_name))
    
    # Parallel processing with progress bar
    results = []
    print(f"\n  Processing {num_chunks} chunks in parallel (max {MAX_WORKERS} at a time)...")
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(split_and_transcribe_chunk, arg): i 
                  for i, arg in enumerate(chunk_args)}
        
        with tqdm(total=num_chunks, desc="  Transcribing", unit="chunk") as pbar:
            for future in as_completed(futures):
                result = future.result()
                results.append(result)
                pbar.update(1)
    
    results.sort(key=lambda x: x['chunk_idx'])
    
    hindi_transcripts = [r['hindi_transcript'] for r in results if r['success']]
    english_transcripts = [r['english_transcript'] for r in results if r['success']]
    
    failed = sum(1 for r in results if not r['success'])
    if failed > 0:
        print(f"\n  ⚠ Warning: {failed} chunks failed")
    
    return hindi_transcripts, english_transcripts

def process_audio_file(audio_path):

    print(f"\n{'='*60}")
    print(f"Processing: {os.path.basename(audio_path)}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    transcripts, needs_chunking = transcribe_audio_direct(audio_path)
    
    if needs_chunking:
        hindi_transcripts, english_transcripts = process_with_chunking(audio_path)
    else:
        hindi_transcripts = [t[0] for t in transcripts]
        english_transcripts = [t[1] for t in transcripts]
    
    hindi_segments = []
    for transcript in hindi_transcripts:
        if hasattr(transcript, 'segments') and transcript.segments:
            for segment in transcript.segments:
                hindi_segments.append({
                    "start": segment['start'],
                    "end": segment['end'],
                    "text": segment['text'].strip()
                })
        elif hasattr(transcript, 'text'):
            hindi_segments.append({"start": 0, "end": 0, "text": transcript.text})
    
   
    english_segments = []
    for transcript in english_transcripts:
        if hasattr(transcript, 'segments') and transcript.segments:
            for segment in transcript.segments:
                english_segments.append({
                    "start": segment['start'],
                    "end": segment['end'],
                    "text": segment['text'].strip()
                })
        elif hasattr(transcript, 'text'):
            english_segments.append({"start": 0, "end": 0, "text": transcript.text})
    
    elapsed = time.time() - start_time
    print(f"\n  ✓ Completed in {elapsed:.1f} seconds")
    print(f"    Hindi: {len(hindi_segments)} segments | English: {len(english_segments)} segments")
    
    return hindi_segments, english_segments

def save_transcription(audio_name, hindi_segments, english_segments):
  
    base_name = os.path.splitext(audio_name)[0]
    
  
    hindi_txt = os.path.join(output_folder, f"{base_name}_HINDI.txt")
    with open(hindi_txt, 'w', encoding='utf-8') as f:
        f.write(f"HINDI TRANSCRIPTION: {audio_name}\n")
        f.write("="*60 + "\n\n")
        for item in hindi_segments:
            m, s = int(item['start']//60), int(item['start']%60)
            em, es = int(item['end']//60), int(item['end']%60)
            f.write(f"[{m:02d}:{s:02d} - {em:02d}:{es:02d}] {item['text']}\n")
    
    # English TXT
    english_txt = os.path.join(output_folder, f"{base_name}_ENGLISH.txt")
    with open(english_txt, 'w', encoding='utf-8') as f:
        f.write(f"ENGLISH TRANSLATION: {audio_name}\n")
        f.write("="*60 + "\n\n")
        for item in english_segments:
            m, s = int(item['start']//60), int(item['start']%60)
            em, es = int(item['end']//60), int(item['end']%60)
            f.write(f"[{m:02d}:{s:02d} - {em:02d}:{es:02d}] {item['text']}\n")
    
    # Combined TXT
    combined_txt = os.path.join(output_folder, f"{base_name}_COMBINED.txt")
    with open(combined_txt, 'w', encoding='utf-8') as f:
        f.write(f"HINDI + ENGLISH: {audio_name}\n")
        f.write("="*60 + "\n\n")
        for i in range(max(len(hindi_segments), len(english_segments))):
            if i < len(hindi_segments):
                item = hindi_segments[i]
                m, s = int(item['start']//60), int(item['start']%60)
                em, es = int(item['end']//60), int(item['end']%60)
                f.write(f"[{m:02d}:{s:02d} - {em:02d}:{es:02d}]\n")
                f.write(f"HINDI: {item['text']}\n")
                if i < len(english_segments):
                    f.write(f"ENGLISH: {english_segments[i]['text']}\n")
                f.write("\n")
    
    print(f"  💾 Saved: {base_name}_HINDI.txt")
    print(f"  💾 Saved: {base_name}_ENGLISH.txt")
    print(f"  💾 Saved: {base_name}_COMBINED.txt")

def main():
   
    print("\n" + "="*60)
    print("⚡ Audio Transcription + Translation (Hindi → English)")
    print("="*60)
    print(f"Settings: Max {MAX_WORKERS} parallel chunks, {CHUNK_LENGTH_MS/1000/60:.0f}min chunks")
    

    audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.mp3')]
    
    if not audio_files:
        print(f"\n✗ No MP3 files in '{audio_folder}' folder!")
        return
    
    print(f"\n✓ Found {len(audio_files)} file(s): {audio_files}\n")
    
    total_start = time.time()
    
    
    for idx, audio_file in enumerate(sorted(audio_files), 1):
        print(f"\n[{idx}/{len(audio_files)}] {audio_file}")
        
        audio_path = os.path.join(audio_folder, audio_file)
        hindi_segments, english_segments = process_audio_file(audio_path)
        
        if hindi_segments and english_segments:
            save_transcription(audio_file, hindi_segments, english_segments)
        else:
            print(f"  ✗ Failed to transcribe {audio_file}")
    
    total_time = time.time() - total_start
    print("\n" + "="*60)
    print(f"🎉 All files completed in {total_time:.1f} seconds!")
    print(f"📁 Results in '{output_folder}' folder")
    print("="*60)

if __name__ == "__main__":
    main()


⚡ Audio Transcription + Translation (Hindi → English)
Settings: Max 5 parallel chunks, 10min chunks

✓ Found 4 file(s): ['video1.mp3', 'video2.mp3', 'video3.mp3', 'video4.mp3']


[1/4] video1.mp3

Processing: video1.mp3
  File size: 36.75 MB
  File too large, will use chunking...
  Loading audio for chunking...
  Duration: 41.8 minutes
  Creating 5 chunks of ~10 minutes each

  Processing 5 chunks in parallel (max 5 at a time)...


  Transcribing:  40%|████      | 2/5 [00:38<01:04, 21.51s/chunk]


  ✗ Chunk 3 failed: Error code: 429 - {'error': {'message': 'Rate limit reached for model `whisper-large-v3` in organization `org_01kf2rwmz8eve8tavv6nj09hhj` service tier `on_demand` on seconds of audio per hour (ASPH): Limit 7200, Used 7139, Requested 600. Please try again in 4m29.5s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'seconds', 'code': 'rate_limit_exceeded'}}


  Transcribing:  60%|██████    | 3/5 [00:46<00:30, 15.20s/chunk]


  ✗ Chunk 1 failed: Error code: 429 - {'error': {'message': 'Rate limit reached for model `whisper-large-v3` in organization `org_01kf2rwmz8eve8tavv6nj09hhj` service tier `on_demand` on seconds of audio per hour (ASPH): Limit 7200, Used 7123, Requested 600. Please try again in 4m21.5s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'seconds', 'code': 'rate_limit_exceeded'}}


...


  ✗ Chunk 5 failed: Error code: 429 - {'error': {'message': 'Rate limit reached for model `whisper-large-v3` in organization `org_01kf2rwmz8eve8tavv6nj09hhj` service tier `on_demand` on seconds of audio per hour (ASPH): Limit 7200, Used 7091, Requested 271. Please try again in 1m21s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'seconds', 'code': 'rate_limit_exceeded'}}


  Transcribing:  83%|████████▎ | 5/6 [00:31<00:03,  3.26s/chunk]


  ✗ Chunk 1 failed: Error code: 429 - {'error': {'message': 'Rate limit reached for model `whisper-large-v3` in organization `org_01kf2rwmz8eve8tavv6nj09hhj` service tier `on_demand` on seconds of audio per hour (ASPH): Limit 7200, Used 7090, Requested 600. Please try again in 4m4.999999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'seconds', 'code': 'rate_limit_exceeded'}}


  Transcribing: 100%|██████████| 6/6 [00:32<00:00,  5.47s/chunk]


  ✗ Chunk 0 failed: Error code: 429 - {'error': {'message': 'Rate limit reached for model `whisper-large-v3` in organization `org_01kf2rwmz8eve8tavv6nj09hhj` service tier `on_demand` on seconds of audio per hour (ASPH): Limit 7200, Used 7087, Requested 600. Please try again in 4m3.5s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'seconds', 'code': 'rate_limit_exceeded'}}

  ⚠ Warning: 6 chunks failed

  ✓ Completed in 43.9 seconds
    Hindi: 0 segments | English: 0 segments
  ✗ Failed to transcribe video4.mp3

🎉 All files completed in 212.9 seconds!
📁 Results in 'transcriptions' folder
